<a href="https://colab.research.google.com/github/AristovYa/First_Test_Repo/blob/main/Lab2_Github2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Подключение библиотек
import pandas as pd     # для чтения и записи табличных файлов
import numpy as np      # для математической обработки данных
import matplotlib.pyplot as plt # для построения графиков
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [2]:
# Получение и чтение Excel файла из облака
url = "https://drive.google.com/uc?export=download&id=130uKatSQ7Tpm4cAED2Q5H5C8WQSdDm8R"
input_df = pd.read_excel(url, decimal=',') 

In [3]:
# Просмотр первых строк таблицы
input_df.head()

,"Wind, m/s"
0,15.18
1,15.84
2,16.60
3,14.98
4,13.28


In [4]:
# Просмотр последних строк таблицы
input_df.tail()

,"Wind, m/s"
715,4.94
716,2.84
717,2.56
718,2.18
719,1.80


In [ ]:
# Строим интерактивный график почасовой скорости ветра при помощи библиотеки Bokeh
wind_speed = input_df.iloc[: , 0].values

output_notebook()

p0 = figure(plot_width = 800, plot_height = 400, title="Почасовая скорость ветра")
p0.line(np.arange(len(wind_speed)), wind_speed, line_width = 2,
       legend_label="Скорость ветра", color = 'green')
show(p0)
# Сохраняем график как csv
np.savetxt("Wind_speed.csv", wind_speed, delimiter=",")

In [ ]:
# Задаём зависимость среднечасовой выработки ВЭС от средней скорости ветра
def wind_speed_to_power(speed):
  min_speed = 5
  max_speed = 20
  critical_speed = 25
  
  # выработка равна нулю, если скорость ветра выше критической или ниже минимальной
  # поэтому обнуляем такую скорость
  speed_tmp = np.where(np.logical_or(speed > critical_speed, speed < min_speed), 0, speed)

  # если скорость выше максимальной, то выработка не будет расти, добавляем ограничение сверху 
  speed_tmp = np.clip(speed_tmp, a_min = None, a_max = max_speed)

  # на рабочем участке от min_speed до max_speed мощность пропорциональна кубу скорости
  power = 0.0017 * (speed_tmp) ** 3
  return power

# Строим интерактивный график почасовой выработки ВЭС от скорости ветра при помощи библиотеки Bokeh
p1 = figure(plot_width = 800, plot_height = 400, title="Почасовой график выработки ВЭС")
p1.line(np.arange(len(wind_speed)), wind_speed_to_power(wind_speed), line_width = 2,
       legend_label="Мощность")
show(p1)

# Сохраняем график как csv
np.savetxt("Wind_speed_to_power_per_hour.csv", wind_speed_to_power(wind_speed), delimiter=",")

In [ ]:
# Рассчитывает посуточные значения выработки ВЭС
temp = wind_speed_to_power(wind_speed).reshape(len(wind_speed)//24, 24)
average_per_day = np.mean(temp, axis=1)

# Строим интерактивный график посуточной выработки ВЭС от скорости ветра при помощи библиотеки Bokeh
p2 = figure(plot_width = 800, plot_height = 400, title="Посуточный график выработки ВЭС")
p2.line(np.arange(len(wind_speed)//24), average_per_day, line_width = 2,
       legend_label="Мощность")
show(p2)

# Сохраняем график как csv
np.savetxt("Wind_speed_to_power_per_day.csv", average_per_day, delimiter=",")